# Save predictions to Pandas

This notebook loads all 2x5 trained models, gets the predictions and saves the results to pandas dataframe and a file

## Setup
- Download and unpack training data
- Create DonkeyCar project and load configuration

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('donkeycar')
sys.path.append('src')

In [ ]:
import os
# do we have the correct env loaded?
assert 'donkey' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [6]:
![ ! -d "data/" ] && \
mkdir data && \
curl -L "https://www.dropbox.com/s/eltelvjt12a4ny1/data.zip?dl=1" --output data/data.zip && \
unzip data/data.zip -d data

In [7]:
![ ! -d "models/" ] && \
mkdir models && \
curl -L "https://www.dropbox.com/s/k9amvpqz3lcu7f3/models.zip?dl=1" --output models/models.zip && \
unzip models/models.zip -d models

In [8]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/


### Common settings for both directions

In [13]:
from dataloader import load_data
import donkeycar as dk
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False
cfg.IMAGE_W = 160
cfg.IMAGE_H = 60

N_FOLDS = 5
DATA_PATH = 'data'
MODELS_PATH = 'models'

model_type = '3d-modified-steering'


loading config file: mycar/config.py
loading personal config over-rides from myconfig.py



### Load Counter Clockwise tubs and get predictions

In [14]:
from dataloader import get_folds
import itertools

DIRECTION = 'CC'

tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))
train_folds, test_folds = get_folds(all_90_speed_data)

from losses import ModelResults

CC_ModelResults = ModelResults(
    model_type,
    MODELS_PATH,
    DIRECTION,
    N_FOLDS,
    tub_records_80_speed,
    tub_records_85_speed,
    tub_records_90_speed,
    train_folds,
    test_folds,
    cfg
)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-1-CC-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-1-CC-80/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-1-CC-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-1-CC-80/catalog_5.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-3-CC-85']


...

Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-3-CC-85/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-2-CC-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-2-CC-90/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-2-CC-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-2-CC-90/catalog_7.catalog


In [15]:
DIRECTION = 'CW'

tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))
train_folds, test_folds = get_folds(all_90_speed_data)

CW_ModelResults = ModelResults(
    model_type,
    MODELS_PATH,
    DIRECTION,
    N_FOLDS,
    tub_records_80_speed,
    tub_records_85_speed,
    tub_records_90_speed,
    train_folds,
    test_folds,
    cfg
)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/2-1-CW-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/2-1-CW-80/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/2-3-CW-85']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/2-3-CW-85/catalog_6.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/3-3-CW-85']
INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/2-2-CW-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/3-3-CW-85/catalog_4.catalog
Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/2-2-CW-90/catalog_2.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/3-2-CW-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/3-2-CW-90/catalog_6.catalog


In [16]:
from donkeycar.utils import get_model_by_type

get_model_by_type(model_type, cfg).interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3


Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_in (InputLayer)          [(None, 3, 120, 160, 3)]  0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 1, 40, 53, 16)     1312      
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 1, 20, 26, 16)     0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 1, 20, 26, 32)     4640      
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 1, 10, 13, 32)     0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 1, 10, 13, 32)     9248      
_________________________________________________________________
flatten_5 (Flatten)          (None, 4160)              0     

In [17]:
CC_ModelResults.predict_results()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


Getting 90-speed  mse-s for fold 1
seq size 3
len ground_truth 4542
36/36 [==============================] - 32s 896ms/step
predictions shape (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.39523977606979055
seq size 3
len ground_truth 5963
11/47 [======>.......................] - ETA: 29s

KeyboardInterrupt: 

In [ ]:
CW_ModelResults.predict_results()

In [15]:

results_unpacked = []

for fold, fold_results in CC_ModelResults.results.items():
    for speed_name, results in fold_results.items():
        for r in results:
            for pred, gt in zip(r.predictions, r.ground_truths):
                results_unpacked.append([r.name, r.direction, r.speed, r.fold, r.is_train, pred, gt])

for fold, fold_results in CW_ModelResults.results.items():
    for speed_name, results in fold_results.items():
        for r in results:
            for pred, gt in zip(r.predictions, r.ground_truths):
                results_unpacked.append([r.name, r.direction, r.speed, r.fold, r.is_train, pred, gt])

len(results_unpacked)

324050

In [16]:
import pandas as pd
df = pd.DataFrame(results_unpacked, columns=['name', 'direction', 'speed', 'fold', 'is_train', 'predicted_angle', 'ground_truth_angle'])


In [17]:
df.to_csv('3D-CNN-model-results.csv.gz', index=False)